# issue
* 사용자사전을 위해 ckonlpy 사용

In [1]:
# import
import seaborn as sns
import matplotlib.pyplot as plt

from tqdm import tqdm,tqdm_notebook  # 진행과정 시각화
tqdm.pandas() #apply사용
from datetime import timedelta  # 시간날짜

import re
import datetime
import pandas as pd
import numpy as np
import gc
import os
import math 

import konlpy
from konlpy.tag import Komoran,Okt,Kkma,Twitter  # 자연어처리
okt = Okt(max_heap_size=5120)

import ckonlpy
tw = ckonlpy.tag.Twitter()
new_noun = pd.read_excel('단어사전.xlsx')['단어'].to_list()
tw.add_dictionary(new_noun,'Noun')

#한글깨짐방지
plt.rc('font',family='Malgun Gothic')
plt.rcParams['axes.unicode_minus'] = False

from IPython.core.display import display, HTML
display(HTML('<style>.container {width:100% !important; }</style>'))

C:\Users\User\Anaconda3\lib\site-packages\jpype\_core.py:217: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)
C:\Users\User\AppData\Roaming\Python\Python36\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [2]:
def tokenized(dataframe, keyword, stopword, batch_size = 10000):
    '''크롤링된 데이터를 불러와 ckonlpy로 토크나이즈 진행
    사용자정의사전을 쉽게 등록할 수 있어 ckonlpy를 사용하여 진행
    
    Parameters
    ---------
    dataframe(DataFrame) : 분석할 df
    keyword(string) : sw에 추가하기 위함
    stopword(list) : 불용어 리스트
    batch_size : 한번에 진행할 row수 (과도하게 늘리면 Java heap memory오류 발생)
    
    Return
    ------
    DataFrame 형태로 './output/token/{keyword}_{start}~{end}.csv' 형태로 저장
    post_dates   year   month  title  full_text     url    Noun        Adjective
    2011-02-31   2011   02     제목   [전체내용]    url    [명사군]    [형용사군]

    '''

    # stopword에 검색어 추가
    stopword.append(keyword.split(' ')[0])
    
    start = 0
    end = len(dataframe)
    
    print(keyword)
    if end<=batch_size:    
        # tqdm.pandas(desc = '토큰화')
        token_df = dataframe.iloc[start:end]['full_text'].progress_apply(lambda x: tw.pos(x, norm=True, stem=True))
        token_df_e = token_df.explode()
        # tqdm.pandas(desc = '단어추출')
        token_noun = token_df_e.apply(lambda x: x[0] if x[1] == 'Noun' else np.nan).dropna()
        token_adj = token_df_e.apply(lambda x: x[0] if x[1] == 'Adjective' else np.nan).dropna()

        # 클리닝
        # tqdm.pandas(desc = '클리닝')
        token_noun = token_noun[token_noun.apply(lambda word: (len(word) > 1) & (word not in sw))]
        token_adj = token_adj[token_adj.apply(lambda word: (len(word) > 1) & (word not in sw))]
        
        #저장
        main_df = pd.merge(dataframe, token_noun.groupby(level=0).agg(list).rename(
            'Noun'), how='left', left_index=True, right_index=True)
        main_df = pd.merge(main_df, token_adj.groupby(level=0).agg(list).rename(
            'Adjective'), how='left', left_index=True, right_index=True)
        main_df.to_csv(f'./output/token/{keyword}_{start}~{end}.csv')
        
    else : #20000개 이상인경우 분할 작업
        final_end = len(dataframe)
        epoch = math.ceil(final_end/batch_size)
        start = 0
        end = batch_size
        while epoch != 0:
            # tqdm.pandas(desc = '토큰화')
            token_df = dataframe.iloc[start:end]['full_text'].progress_apply(lambda x: tw.pos(x, norm=True, stem=True))
            token_df_e = token_df.explode()
            # tqdm.pandas(desc = '단어추출')
            token_noun = token_df_e.apply(lambda x: x[0] if x[1] == 'Noun' else np.nan).dropna()
            token_adj = token_df_e.apply(lambda x: x[0] if x[1] == 'Adjective' else np.nan).dropna()

            # 클리닝
            # tqdm.pandas(desc = '클리닝')
            token_noun = token_noun[token_noun.apply(lambda word: (len(word) > 1) & (word not in sw))]
            token_adj = token_adj[token_adj.apply(lambda word: (len(word) > 1) & (word not in sw))]

            #저장
            main_df = pd.merge(dataframe.iloc[start:end], token_noun.groupby(level=0).agg(list).rename(
                'Noun'), how='left', left_index=True, right_index=True)
            main_df = pd.merge(main_df, token_adj.groupby(level=0).agg(list).rename(
                'Adjective'), how='left', left_index=True, right_index=True)
            os.makedirs(f'./output/token/{keyword}',exist_ok=True)
            main_df.to_csv(f'./output/token/{keyword}/{keyword}_{start}~{end}.csv')
                   
            #next turn
            if epoch >2 :
                start += batch_size
                end += batch_size
            else :      
                start += batch_size
                end = final_end
            
            epoch -= 1
            gc.collect()

### heap 메모리 부족 오류 발생
* heap메모리를 4GB부여하고, 15000개 단위로 분할하여 작업진행 (2만은 가끔 메모리터짐)
* 저장시 기본사항을 붙일수 있게 작업
* 진행이 길어질수록 속도감소(메모리문제로 추정)하기 때문에 재실행필요

In [3]:
#임시 속초/속초해수욕장
sw = list(pd.read_excel("stopword(cp949).xlsx",encoding = 'cp949')['불용어']) #불용어 불러오기
path = 'D:/Python/블로그크롤링/output/크롤링/통합/'
file_list = os.listdir(path)
for file in tqdm_notebook(file_list[:31]):
    file_name = file.split('.')[0]
    main_df = pd.read_csv(path + file)
    keyword = file.split('_')[0]
    tokenized(main_df, keyword, sw)
    gc.collect()

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|                                                                                        | 0/10000 [00:00<?, ?it/s]

강릉 +강원도


  0%|                                                                                | 5/10000 [00:00<03:41, 45.04it/s]

강원도


...

  0%|                                                                                        | 0/10000 [00:00<?, ?it/s]

속초 +강원도


  0%|                                                                                         | 0/2599 [00:00<?, ?it/s]

수산항 +양양


100%|██████████████████████████████████████████████████████████████████████████████| 2599/2599 [02:54<00:00, 14.93it/s]


In [4]:
file_list[:31]

['강릉 +강원도_통합_53544.csv',
 '강원도_통합_61075.csv',
 '광진 +양양_통합_460.csv',
 '광진리 +강원도_통합_398.csv',
 '광진해변 +양양_통합_292.csv',
 '광진해수욕장 +양양_통합_194.csv',
 '기사문 +강원도_통합_990.csv',
 '기사문 +양양_통합_1760.csv',
 '기사문항 +양양_통합_876.csv',
 '기사문해변 +양양_통합_1044.csv',
 '낙산 +강원도_통합_12616.csv',
 '낙산 +양양_통합_14908.csv',
 '낙산항 +양양_통합_336.csv',
 '낙산해수욕장_통합_8083.csv',
 '남애 +양양_통합_2079.csv',
 '남애항 +양양_통합_3130.csv',
 '동산 +양양_통합_1888.csv',
 '동산항 +양양_통합_464.csv',
 '동산해수욕장 +양양_통합_567.csv',
 '동호 +강원도_통합_2523.csv',
 '동호 +양양_통합_2486.csv',
 '동호해변 +양양_통합_1881.csv',
 '동호해수욕장 +양양_통합_725.csv',
 '물치 +강원도_통합_1250.csv',
 '물치 +양양_통합_1852.csv',
 '물치리 +강원도_통합_255.csv',
 '물치항 +양양_통합_2660.csv',
 '설악해수욕장 +강원도_통합_939.csv',
 '설악해수욕장 +양양_통합_1235.csv',
 '속초 +강원도_통합_57030.csv',
 '수산항 +양양_통합_2599.csv']

### check

In [5]:
# file = '광진 +양양_통합_5044.csv'
# file_name = file.split('.')[0]
# main_df = pd.read_csv(path + file)
# keyword = file.split('_')[0]
# tokenized(main_df,file_name,keyword,sw)

In [6]:
file_list

['강릉 +강원도_통합_53544.csv',
 '강원도_통합_61075.csv',
 '광진 +양양_통합_460.csv',
 '광진리 +강원도_통합_398.csv',
 '광진해변 +양양_통합_292.csv',
 '광진해수욕장 +양양_통합_194.csv',
 '기사문 +강원도_통합_990.csv',
 '기사문 +양양_통합_1760.csv',
 '기사문항 +양양_통합_876.csv',
 '기사문해변 +양양_통합_1044.csv',
 '낙산 +강원도_통합_12616.csv',
 '낙산 +양양_통합_14908.csv',
 '낙산항 +양양_통합_336.csv',
 '낙산해수욕장_통합_8083.csv',
 '남애 +양양_통합_2079.csv',
 '남애항 +양양_통합_3130.csv',
 '동산 +양양_통합_1888.csv',
 '동산항 +양양_통합_464.csv',
 '동산해수욕장 +양양_통합_567.csv',
 '동호 +강원도_통합_2523.csv',
 '동호 +양양_통합_2486.csv',
 '동호해변 +양양_통합_1881.csv',
 '동호해수욕장 +양양_통합_725.csv',
 '물치 +강원도_통합_1250.csv',
 '물치 +양양_통합_1852.csv',
 '물치리 +강원도_통합_255.csv',
 '물치항 +양양_통합_2660.csv',
 '설악해수욕장 +강원도_통합_939.csv',
 '설악해수욕장 +양양_통합_1235.csv',
 '속초 +강원도_통합_57030.csv',
 '수산항 +양양_통합_2599.csv',
 '양양 +강원도_통합_47651.csv',
 '오산 +양양_통합_2028.csv',
 '오산리 +양양_통합_1585.csv',
 '오산해수욕장 +양양_통합_842.csv',
 '인구리 +강원도_통합_488.csv',
 '전진리 +강원도_통합_539.csv',
 '죽도 +양양_통합_8355.csv',
 '하광정리 +강원도_통합_634.csv',
 '하조대 +강원도_통합_11099.csv',
 '하조대 +양양_통합_13866.csv',
 '하조대